# Imports

In [51]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore', SyntaxWarning)

from satpy.composites import get_enhanced_image
from satpy.scene import Scene

from datetime import datetime
from glob import glob

import numpy as np

import panel as pn
import hvplot.xarray

## Starting to create satpy scenes

In [ ]:
sat_files = glob("input/G16_ABI-L1b-RadF/*")
sat_files

In [45]:
scn = Scene(filenames = sat_files, reader='abi_l1b')

dataset_names = scn.all_dataset_names()

#print(dataset_names)

In [ ]:
dataset_names

In [47]:
components = scn.available_composite_names()

In [6]:
scn.load([f'C{x:02d}' for x in range(1, 17)])

In [7]:
band_select = pn.widgets.Select(name='Band name', options=dataset_names)

In [8]:
band_select_components = pn.widgets.Select(name='Component name', options=components)

In [9]:
band_select_components

Select(name='Component name', options=['24h_microphysics', ...], value='24h_microphysics')

In [10]:
def get_singleband_plot(bandname=None):
    
    scn.load([bandname])
    result = scn[bandname]
    interactive_plot = result.hvplot.image().opts(title=bandname)

    return interactive_plot

In [11]:
band_iplot = pn.bind(get_singleband_plot,bandname=band_select)
component_iplot = pn.bind(get_singleband_plot, bandname=band_select_components)

In [12]:
template = pn.template.MaterialTemplate(
    #site="",
    title="Visualize GOES-R data",
)
template.main.append(
         pn.Row(
            band_select,
             band_iplot,
             #band_select_components,
             #component_iplot
            #sizing_mode='stretch_both'
            )
        )

In [ ]:
print(scn.available_composite_names())

In [14]:
bandname = 'airmass'
scn.load([bandname])
result = scn[bandname]


In [ ]:
scn.show(bandname)

In [ ]:
scn[bandname].load()

In [ ]:
result

In [ ]:
result.data.plot.imshow()

In [ ]:
da = result.data.load()

In [ ]:
da["bands"] = [0,1,2]

In [ ]:
import numpy as np

In [ ]:
np.floor(da*255)

In [ ]:
np.floor(da*255).hvplot.rgb(x='x',y='y',bands='bands',colorbar=True,rasterize=True,dynamic=True)

In [ ]:
from satpy.composites import get_enhanced_image

result = get_enhanced_image(scn[bandname])


In [19]:
import pandas as pd
import hvplot.pandas
import numpy as np


In [20]:
dt_input = pn.widgets.DatetimeInput(name='Datetime Input',value=datetime(2019, 2, 8))
dt_input

DatetimeInput(name='Datetime Input', type=<class 'datetime.datetime'..., value=datetime.datetime(2019, ...)

In [21]:
def date_print(date=None):
    
    print(date)

    index = pd.date_range('1/1/2000', periods=1000)
    df = pd.DataFrame(np.random.randn(1000, 4), index=index, columns=list('ABCD')).cumsum()
    
    retplot = df.hvplot(title=date.strftime('%Y-%m-%d %H:%M'))

    return retplot
    
dummypane = pn.bind(date_print,date=dt_input)

In [22]:
template = pn.template.MaterialTemplate(
    #site="",
    title="Visualize GOES-R data",
)
# default='%Y-%m-%d %H:%M:%S'
template.main.append(
         pn.Row(
            dt_input,
             dummypane,
            #band_select,
            #band_iplot,
             #band_select_components,
             #component_iplot
            sizing_mode='stretch_both'
            )
        )

2019-02-08 00:00:00


In [23]:
template#.servable()

MaterialTemplate
    [js_area] HTML(None, height=0, margin=0, sizing_mode='fixed', width=0)
    [actions] MaterialTemplateActions()
    [browser_info] BrowserInfo(dark_mode=True, device_pixel_ratio=2, language='en-US', timezone='America/Denver', timezone_offset=360, webdriver=False)
    [busy_indicator] LoadingSpinner(height=20, width=20)
    [main-139833745848912] Row(sizing_mode='stretch_both')
        [0] DatetimeInput(name='Datetime Input', type=<class 'datetime.datetime'..., value=datetime.datetime(2019, ...)
        [1] ParamFunction(function, _pane=HoloViews, defer_load=False)

In [ ]:
import boto3
import botocore
import numpy as np
import pandas as pd
from datetime import datetime
from satpy import Scene
import panel as pn
import hvplot.pandas

# Initialize S3 client with unsigned requests
client = boto3.client('s3', config=botocore.client.Config(signature_version=botocore.UNSIGNED))

# Function to get the latest file based on user input date and time
def get_latest_file(year, day_of_year, hour, time_min):
    while True:
        if hour < 0:
            day_of_year -= 1
            hour = 23
            # Handle the case for the first day of the year
            if day_of_year < 1:
                year -= 1
                day_of_year = 365 if (year % 4 == 0 and (year % 100 != 0 or year % 400 == 0)) else 364
        
        prefix = f'ABI-L1b-RadC/{year}/{day_of_year:03d}/{hour:02d}/'
        # print("Prefix", prefix)
        
        # List objects in the specified path
        result = client.list_objects_v2(Bucket='noaa-goes16', Prefix=prefix)
        # print("Result", result)
        if 'Contents' not in result:
            hour -= 1
            continue
        
        # Extract the minutes from the object keys
        minute_of_observation = np.array([int(obj["Key"].split("_")[3][-5:-3]) for obj in result['Contents']])
        
        # Find the intersection of minutes within the desired range
        intersecting_minutes = np.intersect1d(minute_of_observation, [num for num in range(time_min-4, time_min+1)])
        
        if len(intersecting_minutes) == 0:
            hour -= 1
            time_min = 60  # Reset minute to 60 to start searching the previous hour from the last minute
            continue
        
        # Filter files within the intersecting minutes
        files_within_minute = [obj for obj in result['Contents'] if int(obj["Key"].split("_")[3][-5:-3]) in intersecting_minutes]
        
        # Extract timestamps and get the most recent file
        timestamps_of_files = np.array([obj["LastModified"] for obj in files_within_minute])
        most_recent_file = files_within_minute[timestamps_of_files.argmax()]
        start_time_string = most_recent_file['Key'].split("_")[3]
        file_string = f'ABI-L1b-RadC/{year}/{day_of_year:03d}/{hour:02d}/*_G16_{start_time_string}*/'
        print("file string", file_string)
        filenames = [f's3://noaa-goes16/{file_string}']
        print("filenames", filenames)
        
        return filenames[0]

# Function to handle the input and generate the plot
def handle_date_input(date):
    #input_date = dt_input.value
    print("date", date)
    # Check if the input time is in the future
    current_time = datetime.utcnow()
    if date > current_time:
        output.value = "The input time is in the future. Please enter a valid date and time."
        return
    
    year = date.year
    day_of_year = date.timetuple().tm_yday
    hour = date.hour
    minute = date.minute

    print("year", year)
    print("day" , day_of_year)
    print("hour", hour)
    print("minute",minute)
    
    latest_file = get_latest_file(year, day_of_year, hour, minute)
    print("Latest file", latest_file)
    if not latest_file:
        output.value = "No file found for the specified date and time."
        return
    
    storage_options = {'anon': True}
    scn = Scene(reader='abi_l1b', filenames=[latest_file], reader_kwargs={'storage_options': storage_options})
    scn.load([f'C{x:02d}' for x in range(1, 17)])
    
    return scn
    
dt_input = pn.widgets.DatetimeInput(name='Datetime Input',value=datetime(2024, 2, 8))
    
pn.bind(handle_date_input,date=dt_input)

# Single bands

band_select = pn.widgets.Select(name='Band name', options=dataset_names)

def get_singleband_plot(bandname=None,date=None,return_da=False):
    
    scn = handle_date_input(date)
    scn.load([bandname])
    result = scn[bandname]
    interactive_plot = result.hvplot.image(width=800,height=400,
                                           colorbar=True).opts(title=bandname,
                                                               colorbar_opts = {'title': '%s' %(result.attrs["long_name"]
                                                                                                + " (" +result.attrs["units"]+")")}
                                                              )
    if return_da:
        return result
    return interactive_plot

band_iplot = pn.bind(get_singleband_plot,bandname=band_select,date=dt_input)

# Composite bands

# For composite bands, we have resolution mismatches to get fields
# so we resample all fields to 2km, by using area from the "C13" band


field_select = pn.widgets.Select(name='Composite field name', options=scn.available_composite_names())

def get_compositebands_plot(fieldname=None,date=None):
    
    scn = handle_date_input(date)
    resampling_area = scn["C13"].area
    resampled_scn = scn.resample(resampling_area)

    
    resampled_scn.load([fieldname])
    
    crs = resampled_scn[fieldname].attrs['area'].to_cartopy_crs()
    
    result = get_enhanced_image(resampled_scn[fieldname])

    result_da = result.data

    #scale result data array to 0 to 255, as 0 to 1 does not seem to work with hvplot
    result_da = result_da*255
    
    interactive_plot = result_da.hvplot.rgb(x='x',y='y',bands='bands',colorbar=True,
                                            #geo=True,crs=crs,
                         rasterize=True,dynamic=True,width=800,height=400).opts(title=fieldname)

    return interactive_plot

field_iplot = pn.bind(get_compositebands_plot,fieldname=field_select,date=dt_input)

# App layout

template = pn.template.MaterialTemplate(
    site="Pythia cookbook",
    title="Visualize GOES-R data",
    sidebar= [dt_input],
    #collapsed_sidebar=True
)

template.main.append(
    pn.Tabs(
        ("Single fields",
         pn.Row(
            band_select,
             band_iplot
            #sizing_mode='stretch_both'
            ),
        ),
        ("Composite fields",   
         pn.Row(
             field_select,
             field_iplot
            #sizing_mode='stretch_both'
            )
        )
    )
)

In [ ]:
template#.servable()
